### NIBRS Data
An incident can have many different "offenses" attached to it (i.e., Robbery, Motor Vehicle Theft). For each offense, it can have many different offenders, arrestees, victims, and property mentioned. 

To account for this, the dataset is separable into 6 combinations, and a full join on everything would result in offenses with multiple arrestees being listed twice.

Virginia first started using the NIBRS system in 1994; however, it was still up to each individual police department to report crimes in a format they want. 

Two goals with NIBRS data:
- Are we consistent with police departments we include?
- Are we still representative of the state by population and demographics (rural, urban)?

In [ ]:
import requests